In [1]:
import pandas as pd
from pyvi import ViUtils

In [2]:
cps_df = pd.read_csv('../data/cellphones.csv')
for column in cps_df.columns:
    print(column)

name
new_price 
old_price
kích thước màn hình
công nghệ màn hình
camera sau
camera trước
chipset
dung lượng RAM
bộ nhớ trong
pin
thẻ SIM
hệ điều hành
tính năng màn hình
lõi CPU
trọng lượng
bluetooth
tần số quét


In [3]:
drop_columns = ['old_price', 'lõi CPU', 'tính năng màn hình', 'trọng lượng']

for column in drop_columns:
    del cps_df[column]

cps_df.head()

,name,new_price,kích thước màn hình,công nghệ màn hình,camera sau,camera trước,chipset,dung lượng RAM,bộ nhớ trong,pin,thẻ SIM,hệ điều hành,bluetooth,tần số quét
0,iPhone 11 64GB I Chính hãng VN/A,11.690.000 ₫,6.1 inches,IPS LCD,Camera kép 12MP:\n- Camera góc rộng: ƒ/1.8 ape...,"12 MP, ƒ/2.2 aperture",A13 Bionic,4 GB,64 GB,3110 mAh,Nano-SIM + eSIM,iOS 13 hoặc cao hơn (Tùy vào phiên bản phát hành),5,60Hz
1,iPhone 13 Pro Max 128GB | Chính hãng VN/A,29.690.000 ₫,6.7 inches,OLED,"Camera góc rộng: 12MP, ƒ/1.5\nCamera góc siêu ...","12MP, ƒ/2.2",Apple A15,6 GB,128 GB,"4,325mAh",2 SIM (nano‑SIM và eSIM),iOS15,v5.0,120Hz
2,iPhone 11 Pro Max Chính hãng\nVN/A,26.000.000 ₫,6.5 inches,OLED,3 Camera 12MP:\n- Camera tele: ƒ/2.0 aperture\...,12 MP ƒ/2.2 aperture,A13 Bionic,4 GB,64 GB,3969 mAh,Nano-SIM + eSIM,iOS 13 hoặc cao hơn (Tùy vào phiên bản phát hành),5,NaN
3,iPhone 12 Pro Max 128GB I Chính hãng VN/A,27.000.000 ₫,6.7 inches,OLED,"Camera chính: 12 MP, f/1.6, 26mm, 1.4µm, dual ...","12 MP, f/2.2, 23mm (wide), 1/3.6""\nSL 3D, (dep...",Apple A14 Bionic (5 nm),6 GB,128 GB,"Li-Ion, sạc nhanh 20W, sạc không dây 15W, USB ...",2 SIM (nano‑SIM và eSIM),iOS 14.1 hoặc cao hơn (Tùy vào phiên bản phát ...,"5.0, A2DP, LE",60Hz
4,iPhone 13 128GB | Chính hãng VN/A,21.200.000 ₫,6.1 inches,OLED,"Camera góc rộng: 12MP, f/1.6\nCamera góc siêu ...","12MP, f/2.2",Apple A15,4 GB,128 GB,Khoảng 3.200mAh,NaN,iOS 15,v5.0,60Hz


# Xử lý tên cột

In [4]:
cps_df = cps_df.rename(columns={"name": "ten", "new_price ": "gia_moi"})

for column in cps_df.columns:
    cps_df = cps_df.rename(columns={column: ViUtils.remove_accents(column).decode("utf-8").lower().strip().replace(' ', '_')})

# Xử lý trường tên

In [5]:
cps_df = cps_df[cps_df['ten'].notna()]
cps_df['ten'] = cps_df['ten'].apply(lambda s: str(s).replace('|', '').replace('Chính hãng', '').replace('VN/A', '').replace('I', '').replace('Điện thoại', '').strip())
cps_df['ten'] = cps_df['ten'].apply(lambda s: ViUtils.remove_accents(s).decode("utf-8"))

In [6]:
cps_df = cps_df[cps_df['gia_moi'].notna()]
cps_df['gia_moi'] = cps_df['gia_moi'].apply(lambda s: str(s).replace('₫', '').replace('.', '').replace('*', '').strip())
cps_df['gia_moi'] = cps_df['gia_moi'].apply(lambda s: int(s))

In [7]:

cps_df = cps_df[cps_df['kich_thuoc_man_hinh'].notna()]

In [8]:
def process_screen_size(s):
    size = s.split(' - ')[0]
    if "&" in s:
        size = size.split(' & ')
        size1 = float(size[0].split()[1])
        size2 = float(size[1].split()[1])
        size = size1 + size2
    else:
        size = float(size.split()[0])
    return size

cps_df['kich_thuoc_man_hinh'] = cps_df['kich_thuoc_man_hinh'].apply(process_screen_size)

In [9]:
def process_screen_frequency(s):
    s = str(s)
    if s == 'nan':
        f = 0
    elif ' ' in s:
        f = s.split()[1][:-2]
    else:
        f = s[:-2]
    return int(f)

cps_df['tan_so_quet'] = cps_df['tan_so_quet'].apply(process_screen_frequency)

In [10]:
def process_rom(s):
    try:
        if 'GB' in s:
            return float(s[:-3])
        elif 'MB' in s:
            return float(s[:-3])/1024
        else:
            return float(s[:-3])*1024
    except:
        return 0

cps_df['bo_nho_trong'] = cps_df['bo_nho_trong'].apply(process_rom)

In [11]:
cps_df.head()

,ten,gia_moi,kich_thuoc_man_hinh,cong_nghe_man_hinh,camera_sau,camera_truoc,chipset,dung_luong_ram,bo_nho_trong,pin,the_sim,he_dieu_hanh,bluetooth,tan_so_quet
0,iPhone 11 64GB,11690000,6.1,IPS LCD,Camera kép 12MP:\n- Camera góc rộng: ƒ/1.8 ape...,"12 MP, ƒ/2.2 aperture",A13 Bionic,4 GB,64.0,3110 mAh,Nano-SIM + eSIM,iOS 13 hoặc cao hơn (Tùy vào phiên bản phát hành),5,60
1,iPhone 13 Pro Max 128GB,29690000,6.7,OLED,"Camera góc rộng: 12MP, ƒ/1.5\nCamera góc siêu ...","12MP, ƒ/2.2",Apple A15,6 GB,128.0,"4,325mAh",2 SIM (nano‑SIM và eSIM),iOS15,v5.0,120
2,iPhone 11 Pro Max,26000000,6.5,OLED,3 Camera 12MP:\n- Camera tele: ƒ/2.0 aperture\...,12 MP ƒ/2.2 aperture,A13 Bionic,4 GB,64.0,3969 mAh,Nano-SIM + eSIM,iOS 13 hoặc cao hơn (Tùy vào phiên bản phát hành),5,0
3,iPhone 12 Pro Max 128GB,27000000,6.7,OLED,"Camera chính: 12 MP, f/1.6, 26mm, 1.4µm, dual ...","12 MP, f/2.2, 23mm (wide), 1/3.6""\nSL 3D, (dep...",Apple A14 Bionic (5 nm),6 GB,128.0,"Li-Ion, sạc nhanh 20W, sạc không dây 15W, USB ...",2 SIM (nano‑SIM và eSIM),iOS 14.1 hoặc cao hơn (Tùy vào phiên bản phát ...,"5.0, A2DP, LE",60
4,iPhone 13 128GB,21200000,6.1,OLED,"Camera góc rộng: 12MP, f/1.6\nCamera góc siêu ...","12MP, f/2.2",Apple A15,4 GB,128.0,Khoảng 3.200mAh,NaN,iOS 15,v5.0,60


In [12]:
cps_df.to_csv('../clean_data/cellphones.csv', index=False)